# Número óptimo de Tópicos con métrica Coherence

In [1]:
import sys
sys.path.insert(0, '..')
from utils.cargar import df_caso
from utils.preprocesamiento import process_df, procesar_adela, StemmerTokenizer
from sklearn.model_selection import train_test_split

tokenizador = StemmerTokenizer(stem=False,rmv_punctuation=True)

caso = 'adela'

df = df_caso(caso)
df = procesar_adela(df)
df = df[df['opt_left'] == 'Producir el alimento contra déficit vitamínico']
df = process_df(df,'comment','sel',verbose=True)
df = df.drop(columns=['user_id','team_id','gender','df','title','opt_left','opt_right','max_num','phase','time','curso'])

df_train, df_test, _, _ = train_test_split(df, df['sel'], test_size=.05, stratify=df['sel'], random_state=0)
tokenized_corpus = [tokenizador(document) for document in df_train['comment']]
tokenized_test = [tokenizador(document) for document in df_test['comment']]

26 rows found with non string elements for column comment (0.40%)
Deleting 685 columns for which max target value is over 7 (10.58%)
5761 available rows after processing


In [2]:
import gensim
from gensim import corpora

# Create a dictionary from the tokenized corpus
dictionary = corpora.Dictionary(tokenized_corpus)

# Convert the tokenized corpus into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_corpus]

In [3]:
from gensim.models import CoherenceModel

# Set the range of topic numbers to try
min_topics = 2
max_topics = 10
step_size = 1

# Initialize variables for best coherence score and best number of topics
best_coherence_score = -1
best_num_topics = -1

# Iterate over the range of topic numbers
for num_topics in range(min_topics, max_topics+1, step_size):
    # Train the LDA model
    lda_model = gensim.models.LdaModel(doc_term_matrix, num_topics=num_topics, id2word=dictionary, passes=10)
    
    # Calculate coherence score
    coherence_model = CoherenceModel(model=lda_model, texts=tokenized_corpus, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    
    # Check if coherence score is the best so far
    if coherence_score > best_coherence_score:
        best_coherence_score = coherence_score
        best_num_topics = num_topics

# Print the best number of topics
print(f"Best number of topics: {best_num_topics}")

Best number of topics: 4


In [4]:
%%time
lda_model_opt = gensim.models.LdaModel(doc_term_matrix, num_topics=best_num_topics, id2word=dictionary, passes=10)

CPU times: user 16.7 s, sys: 15.9 ms, total: 16.7 s
Wall time: 16.9 s


In [5]:
# Print the generated topics
topics = lda_model_opt.print_topics(num_topics=num_topics)
for topic in topics:
    print(topic)

(0, '0.022*"tradiciones" + 0.015*"alimento" + 0.014*"pueblo" + 0.013*"si" + 0.012*"importante" + 0.012*"pueblos" + 0.011*"puede" + 0.011*"acuerdo" + 0.010*"llegar" + 0.010*"mas"')
(1, '0.019*"pueblo" + 0.015*"fruta" + 0.013*"producción" + 0.012*"debe" + 0.011*"proyecto" + 0.010*"tradiciones" + 0.009*"comunidad" + 0.009*"debería" + 0.008*"puede" + 0.008*"Adela"')
(2, '0.031*"tradiciones" + 0.029*"alimento" + 0.024*"pueblo" + 0.017*"importante" + 0.015*"personas" + 0.015*"producir" + 0.013*"originario" + 0.011*"producción" + 0.011*"llegar" + 0.011*"acuerdo"')
(3, '0.028*"tradiciones" + 0.018*"vitamina" + 0.012*"D" + 0.012*"salud" + 0.009*"si" + 0.008*"importante" + 0.008*"pueblo" + 0.008*"personas" + 0.007*"fruta" + 0.007*"población"')


In [6]:
print(' '.join(tokenized_test[0]) + '\n')

# Convert the tokenized document into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(tokenized_test[0])]

# Get the topic probabilities for the new document
topic_probs = lda_model_opt.get_document_topics(doc_term_matrix)[0]

# Print the topic probabilities
for topic, prob in topic_probs:
    print(f"Topic {topic}: {prob}")

Se debe priorizar respetar gente pueblo insistir cantidad gente beneficiaria fruta llegar acuerdo beneficie bandos problema

Topic 0: 0.01584179513156414
Topic 1: 0.23093238472938538
Topic 2: 0.7381098866462708
Topic 3: 0.015115933492779732
